In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from IPython.display import display, HTML
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import datasets
from scipy import stats

In [2]:
def confusion_table(confusion_mtx):
    """Renders a nice confusion table with labels"""
    confusion_df = pd.DataFrame({'y_pred=0': np.append(confusion_mtx[:, 0], confusion_mtx.sum(axis=0)[0]),
                                 'y_pred=1': np.append(confusion_mtx[:, 1], confusion_mtx.sum(axis=0)[1]),
                                 'Total': np.append(confusion_mtx.sum(axis=1), ''),
                                 '': ['y=0', 'y=1', 'Total']}).set_index('')
    return confusion_df

def total_error_rate(confusion_matrix):
    """Derive total error rate from confusion matrix"""
    return 1 - np.trace(confusion_mtx) / np.sum(confusion_mtx)

In [3]:
default_df = pd.read_csv('../data/Default.csv', index_col='Unnamed: 0')
default_df = default_df.reset_index().drop('index', axis=1)

# Check for missing
assert default_df.isna().sum().sum() == 0

# Rationalise types
default_df = pd.get_dummies(default_df, dtype=np.float64).drop(['default_No', 'student_No'], axis=1)

display(default_df.head())

,balance,income,default_Yes,student_Yes
0,729.526495,44361.625074,0.0,0.0
1,817.180407,12106.134700,0.0,1.0
2,1073.549164,31767.138947,0.0,0.0
3,529.250605,35704.493935,0.0,0.0
4,785.655883,38463.495879,0.0,0.0


In [4]:
# c
# Repeat the process in (b) three times, 
# using three different splits of the observations into a training 
# set and a validation set. Com- ment on the results obtained.
for s in range(1,4):
    display(HTML('<h3>Random seed = {}</h3>'.format(s)))
    # Create index for 50% holdout set
    np.random.seed(s)
    train = np.random.rand(len(default_df)) < 0.5
    
    response   = 'default_Yes'
    predictors = ['income', 'balance']
    
    X_train = np.array(default_df[train][predictors])
    X_test  = np.array(default_df[~train][predictors])
    y_train = np.array(default_df[train][response])
    y_test  = np.array(default_df[~train][response])
    
    # Logistic regression
    logit       = LogisticRegression()
    model_logit = logit.fit(X_train, y_train)
    
    # Predict
    y_pred = model_logit.predict(X_test)
    
    # Analysis
    confusion_mtx = confusion_matrix(y_test, y_pred)
    display(confusion_table(confusion_mtx))
    
    total_error_rate_pct = np.around(total_error_rate(confusion_mtx) * 100, 4)
    print('total_error_rate: {}%'.format(total_error_rate_pct))

,y_pred=0,y_pred=1,Total
,,,
y=0,4846,0,4846
y=1,164,0,164
Total,5010,0,


total_error_rate: 3.2735%


,y_pred=0,y_pred=1,Total
,,,
y=0,4691,1,4692
y=1,176,0,176
Total,4867,1,


total_error_rate: 3.636%


,y_pred=0,y_pred=1,Total
,,,
y=0,4773,1,4774
y=1,163,0,163
Total,4936,1,


total_error_rate: 3.3219%


In [5]:
#d 
# Now consider a logistic regression model that predicts the 
# probability of default using income, balance, and a dummy variable for student. 
# Estimate the test error for this model using the validation set approach. 
# Comment on whether or not including a dummy variable for student 
# leads to a reduction in the test error rate.

for s in range(1,4):
    display(HTML('<h3>Random seed = {}</h3>'.format(s)))
    # Create index for 50% holdout set
    np.random.seed(s)
    train = np.random.rand(len(default_df)) < 0.5
    
    response   = 'default_Yes'
    predictors = ['income', 'balance', 'student_Yes']
    
    X_train = np.array(default_df[train][predictors])
    X_test  = np.array(default_df[~train][predictors])
    y_train = np.array(default_df[train][response])
    y_test  = np.array(default_df[~train][response])
    
    # Logistic regression
    logit       = LogisticRegression()
    model_logit = logit.fit(X_train, y_train)
    
    # Predict
    y_pred = model_logit.predict(X_test)
    
    # Analysis
    confusion_mtx = confusion_matrix(y_test, y_pred)
    display(confusion_table(confusion_mtx))
    
    total_error_rate_pct = np.around(total_error_rate(confusion_mtx) * 100, 4)
    print('total_error_rate: {}%'.format(total_error_rate_pct))

,y_pred=0,y_pred=1,Total
,,,
y=0,4846,0,4846
y=1,164,0,164
Total,5010,0,


total_error_rate: 3.2735%


,y_pred=0,y_pred=1,Total
,,,
y=0,4691,1,4692
y=1,176,0,176
Total,4867,1,


total_error_rate: 3.636%


,y_pred=0,y_pred=1,Total
,,,
y=0,4773,1,4774
y=1,163,0,163
Total,4936,1,


total_error_rate: 3.3219%


In [13]:
# 9 We will now consider the Boston 
# housing data set, from the MASS library.

boston = datasets.load_boston()
boston_feat = pd.DataFrame(boston.data, columns=boston.feature_names)
boston_resp = pd.Series(boston.target).rename('medv')

boston_df = pd.concat([boston_feat, boston_resp], axis=1)

# Check for missing values
#assert boston_df.isnull().sum().sum() == 0

boston_df.head()

/Users/namnguyen/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets 

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,medv
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [14]:
# (a) Based on this data set, provide an estimate 
# for the population mean of medv. Call this estimate μˆ

mu_hat = boston_df['medv'].mean()
display(mu_hat)

22.532806324110677

In [15]:
# (b) Provide an estimate of the standard error of μˆ. 
# Interpret this result.

def standard_error_mean(df):
    """Compute estimated standard error of the mean
    with analytic approach"""
    medv = np.array(df)
    SE   = np.std(medv) / np.sqrt(len(medv))
    return SE
display(standard_error_mean(boston_df['medv']))

0.4084569346972866

In [16]:
# Compute standard error of the mean with the bootstrap approach

def mean_boot(df, idx):
    Z = np.array(df.loc[idx])
    return np.mean(Z)

def boot_idx(n):
    """Return index for bootstrap sample of size n
    e.g. generate array in range 0 to n, with replacement"""
    return np.random.randint(low=0, high=n, size=n)

def boot(fn, data_df, samples):
    """Perform bootstrap for B number of samples"""
    results = []
    for s in range(samples):
        Z = fn(data_df, boot_idx(data_df.shape[0]))
        results += [Z]
    return np.array(results)

B = 10000
mean_boot  = boot(mean_boot, boston_df['medv'], samples=B)
SE_pred    = np.std(mean_boot) 

print('SE: ' + str(SE_pred))

SE: 0.415929612256344


In [17]:
#(d) Based on your bootstrap estimate from (c), 
# provide a 95 % confidence interval for the mean of medv. 
# Compare it to the results obtained using

mu_hat   = np.mean(boston_df['medv'])
conf_low = mu_hat - (2*SE_pred)
conf_hi  = mu_hat + (2*SE_pred)

pd.Series({'mu': mu_hat, 
           'SE': SE_pred,
           '[0.025': conf_low,
           '0.975]': conf_hi})

mu        22.532806
SE         0.415930
[0.025    21.700947
0.975]    23.364666
dtype: float64

In [ ]:
#(e) Based on this dataset, provide an estimate, μˆmed, 
# for the median value of medv in the population.

In [18]:
median_hat = np.median(boston_df['medv'])
print('median: ' + str(median_hat))

median: 21.2


In [20]:
#g) Based on this data set, 
# provide an estimate for the tenth percentile of medv 
# in Boston suburbs. Call this quantity μˆ0.1. 

tenth_percentile = np.percentile(boston_df['medv'], 10)
print('tenth_percentile: ' + str(tenth_percentile))

tenth_percentile: 12.75


In [21]:
# (h) Use the bootstrap to estimate the standard error of μˆ0.1. 
# Comment on your findings.
# Compute standard error of the tenth percentile with the 
# bootstrap approach

def tenth_percentile(df, idx):
    Z = np.array(df.loc[idx])
    return np.percentile(Z, 10)

def boot_idx(n):
    """Return index for bootstrap sample of size n
    e.g. generate array in range 0 to n, with replacement"""
    return np.random.randint(low=0, high=n, size=n)

def boot(fn, data_df, samples):
    """Perform bootstrap for B number of samples"""
    results = []
    for s in range(samples):
        Z = fn(data_df, boot_idx(data_df.shape[0]))
        results += [Z]
    return np.array(results)

B = 10000
boot_obs   = boot(tenth_percentile, boston_df['medv'], samples=B)
SE_pred    = np.std(boot_obs) 

print('SE: ' + str(SE_pred))

SE: 0.503832996041347


## 5.3.1 The Validation Set Approach

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from collections import OrderedDict

In [3]:
Auto = pd.read_csv('../data/Auto.csv', header=0, na_values='?')
Auto = Auto.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
print(Auto.shape)
print(Auto.head())

(392, 9)
    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0       130.0    3504          12.0    70   
1  15.0          8         350.0       165.0    3693          11.5    70   
2  18.0          8         318.0       150.0    3436          11.0    70   
3  16.0          8         304.0       150.0    3433          12.0    70   
4  17.0          8         302.0       140.0    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


In [5]:
# split the data into training and record the index of train samples
np.random.seed(1)
train = np.random.choice(Auto.shape[0], 196, replace=False)
select = np.in1d(range(Auto.shape[0]), train) #mask

In [7]:
# start to build the model
lm = smf.ols ('mpg~horsepower', data = Auto[select]).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Mon, 14 Nov 2022   Prob (F-statistic):           1.28e-42
Time:                        14:21:12   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

In [8]:
# prediction values
preds = lm.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('-------- Test error for 1st order model --------')
print(np.mean(square_error[~select])) # "~" excludes values from training samples

-------- Test error for 1st order model --------
23.361902892587235


In [9]:
# build a model with 2nd order of features  
lm2 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0)', data = Auto[select]).fit()
preds = lm2.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('--------Test error for 2nd order--------')
print(square_error[~select].mean())

--------Test error for 2nd order--------
20.25269085835007


In [10]:
lm3 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)', data = Auto[select]).fit()
preds = lm3.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('--------Test rror for 3rd order--------')
print(np.mean(square_error[~select]))

# There is not much improvement in cubic model over the quadratic model.

--------Test rror for 3rd order--------
20.325609365773644


## 5.3.2 Leave-One-Out Cross-Validation

In [12]:
# OLS fit 
ols_fit = smf.ols ('mpg~horsepower', data = Auto).fit()
print(ols_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [13]:
# GLM fit. Compare with OLS fit, the coeffs are the same
glm_fit = smf.glm('mpg~horsepower', data = Auto).fit()
print(glm_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [14]:
# let us re-train the model in sklearn
x = pd.DataFrame(Auto.horsepower)
y = Auto.mpg

# to use some of implemented function in Python, we use Sklearn for linear model 
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.pipeline import Pipeline

model = LinearRegression()
model.fit(x, y)
print(model.intercept_)
print(model.coef_)

39.93586102117047
[-0.15784473]


In [15]:
# loo use folds equal to # of observations. We could also choose other number of folds.
k_fold = KFold(n_splits=x.shape[0]) 
test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
print(np.mean(-test))

24.231513517929226


In [16]:
# for higher order polynomial fit, we use pipline tool. 
# below shows how to fit an order 1 to 20 polynomial data and show the loo results
# this step may take a few mins
A = OrderedDict()
n_split = x.shape[0]
for porder in range(1, 21, 2):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=n_split) # loo use folds equal to # of observations
    test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
    A[str(porder)] = np.mean(-test)
    
print(A) # MSE for each order of polynomial

OrderedDict([('1', 24.231513517929226), ('3', 19.334984064118647), ('5', 19.033206870017146), ('7', 19.125446846687602), ('9', 19.13392619499544), ('11', 19.136033100530025), ('13', 27.76341651771519), ('15', 35.29333367792178), ('17', 43.65451056724629), ('19', 60.96829353029924)])


## 5.3.3 k-Fold Cross-Validation

In [17]:
# K-fold validation is exactly same as LOO with different n_splits parameter setup. 
# the computation time is much shorter than that of LOOCV.
np.random.seed(2)
A = OrderedDict()
n_split = 10 # vs x.shape[0]
for porder in range(1, 21, 2):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=n_split) 
    test = cross_val_score(model, x, y, cv = k_fold,  scoring = 'neg_mean_squared_error', n_jobs = -1)
    A[str(porder)] = np.mean(-test)
    
print(A)

OrderedDict([('1', 27.439933652339864), ('3', 21.33660618337527), ('5', 20.905574966439268), ('7', 20.95311639832179), ('9', 21.035267386079106), ('11', 21.454630737710676), ('13', 30.81007847852421), ('15', 39.536744276345885), ('17', 48.27499367526184), ('19', 64.02367508039754)])


## 5.3.4 The Bootstrap
resampling

In [18]:
Portfolio = pd.read_csv('../data/Portfolio.csv', header=0)

In [19]:
def alpha_fn(data, index):
    X = data.X.iloc[index]
    Y = data.Y.iloc[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

alpha_fn(Portfolio, range(0,100))

0.5766511516104116

In [20]:
# generate one set of random index with 100 elements. The array has been sorted to show there are repeat elements.
np.sort(np.random.choice(range(0, 100), size=100, replace=True))

# recall the previous function with a random set of input. 
alpha_fn(Portfolio, np.random.choice(range(0, 100), size=100, replace=True))

def boot_python(data, input_fun, iteration):
    n = Portfolio.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)
    for i in range(len(idx)):
        stat[i] = input_fun(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}

In [21]:
boot_python(Portfolio, alpha_fn, 1000)

{'Mean': 0.5811900883897445, 'STD': 0.09408713019844589}